In [1]:
import os
import numpy as np
import pandas as pd
from pytorch_tabnet.tab_model import TabNetClassifier
from sklearn.metrics import roc_auc_score

#Import data
train = pd.read_csv('/home/nghia/PR_LAB/KERC_Chall/kerc2021/KERC21Dataset/KERC21Dataset/train_Tab.csv')
train['label']

0      LALV
1      HALV
2      HALV
3      LALV
4      LALV
       ... 
685    LAHV
686    LAHV
687    HALV
688    LAHV
689    LAHV
Name: label, Length: 690, dtype: object

In [2]:
import torch
from sklearn.preprocessing import LabelEncoder

nunique = train.nunique()
types = train.dtypes

categorical_columns = []
categorical_dims = {}

for col in train.columns:
    l_enc = LabelEncoder()
    train[col] = l_enc.fit_transform(train[col].values)
    categorical_columns.append(col)
    categorical_dims[col] = len(l_enc.classes_)

train['label']

0      3
1      1
2      1
3      3
4      3
      ..
685    2
686    2
687    1
688    2
689    2
Name: label, Length: 690, dtype: int64

In [3]:
fea_list = [x for x in train.columns.to_list() if x not in ["ID", "label"]]
features = [col for col in train[fea_list]]
cat_idxs = [i for i, f in enumerate(features) if f in categorical_columns]
cat_dims = [ categorical_dims[f] for i, f in enumerate(features) if f in categorical_columns]
print(len(cat_dims), len(cat_idxs))

316 316


In [5]:
#Tabnet classifier
clf = TabNetClassifier(cat_idxs= cat_idxs,
                       cat_dims= cat_dims,
                       cat_emb_dim=1,
                       optimizer_fn=torch.optim.Adam,
                       optimizer_params=dict(lr=2e-2),
                       scheduler_params={"step_size":50,
                                         "gamma":0.9},
                       scheduler_fn=torch.optim.lr_scheduler.StepLR,
                       mask_type= "sparsemax", #'entmax',
                       #device_name='cpu'
                       )
X_train = train[features].values
y_train = train["label"].values

max_epochs = 100 if not os.getenv("CI", False) else 2
#clf.fit(
 #   X_train, y_train, #X_train=X_train, y_train=y_train,
    #eval_set=[(X_train, y_train), (X_val, y_val)],
    #eval_name=['train', 'valid'],
    #eval_metric=['auc'],
  #  max_epochs=max_epochs, patience=20,
   # batch_size=1024, virtual_batch_size=128,
   # num_workers=0,
    #weights=1,
    #drop_last=False
#)


Device used : cuda


In [6]:
val = pd.read_csv('/home/nghia/PR_LAB/KERC_Chall/kerc2021/KERC21Dataset/KERC21Dataset/val_Tab.csv')
import torch
from sklearn.preprocessing import LabelEncoder

nunique = val.nunique()
types = val.dtypes

categorical_columns = []
categorical_dims = {}

for col in val.columns:
    l_enc = LabelEncoder()
    val[col] = l_enc.fit_transform(val[col].values)
    categorical_columns.append(col)
    categorical_dims[col] = len(l_enc.classes_)

fea_val_list = [x for x in val.columns.to_list() if x not in ["ID"]]
features = [col for col in val[fea_val_list]]
cat_idxs = [i for i, f in enumerate(features) if f in categorical_columns]
cat_dims = [ categorical_dims[f] for i, f in enumerate(features) if f in categorical_columns]

In [7]:
print(len(cat_dims), len(cat_idxs))


316 316


In [8]:
X_val = val[features].values
#y_val = val["label"].values


In [9]:
y_pred = clf.predict(X_val)
y_pred

array([1, 1, 3, 3, 1, 1, 0, 2, 1, 0, 2, 3, 3, 3, 3, 2, 1, 3, 1, 3, 3, 3,
       3, 3, 3, 1, 1, 3, 3, 0, 3, 1, 3, 1, 1, 3, 2, 1, 1, 3, 3, 2, 3, 3,
       1, 1, 0, 1, 3, 1, 1, 1, 1, 0, 3, 3, 1, 3, 3, 1, 1, 1, 0, 1, 3, 3,
       2, 3, 2, 1, 1, 3, 1, 1, 3, 3, 3, 3, 3, 3, 3, 2, 3, 3, 1, 1, 1, 3,
       1, 3, 3, 1, 3, 1, 3, 1, 3, 3, 1, 3, 3, 3, 1, 3, 1, 3, 1, 2, 0, 0,
       3, 3, 2, 3, 3, 3, 1, 3, 3, 3, 2, 3, 3, 3, 3, 3, 3, 3, 3, 0, 1, 1,
       1, 1, 3, 3, 3, 3, 2, 1, 3, 1, 1, 3, 1, 1, 3, 3, 3, 1, 1, 1, 3, 2,
       1, 1, 3, 3, 1, 1, 1, 1, 1, 3, 0, 1, 3, 1, 3, 3, 3, 2, 1, 3, 1, 3,
       1, 3, 3, 3, 3, 0, 3, 3, 3, 3, 1, 1, 1, 3, 3, 1, 3, 0, 3, 1, 3, 0,
       1, 3, 2, 0, 3, 3, 3, 1, 3, 3, 3, 0, 1, 3, 0, 3, 3, 1, 0, 0, 3, 1,
       3, 3, 3, 3, 1, 3, 3, 1, 1, 1, 1, 1, 1, 3])

In [8]:
test = pd.read_csv('/home/nghia/PR_LAB/KERC_Chall/kerc2021/KERC21Dataset/KERC21Dataset/test_Tab.csv')
nunique = test.nunique()
types = test.dtypes

categorical_columns = []
categorical_dims = {}

for col in test.columns:
    l_enc = LabelEncoder()
    test[col] = l_enc.fit_transform(test[col].values)
    categorical_columns.append(col)
    categorical_dims[col] = len(l_enc.classes_)

fea_test_list = [x for x in test.columns.to_list() if x not in ["ID"]]
features = [col for col in test[fea_test_list]]
cat_idxs = [i for i, f in enumerate(features) if f in categorical_columns]
cat_dims = [ categorical_dims[f] for i, f in enumerate(features) if f in categorical_columns]

In [11]:
X_test = test[features].values

y_pred_test = clf.predict(X_test)
y_pred_test

array([3, 1, 1, 3, 1, 3, 1, 1, 3, 1, 3, 1, 1, 3, 1, 1, 3, 3, 3, 1, 3, 1,
       3, 3, 3, 3, 3, 3, 2, 1, 3, 1, 1, 1, 3, 3, 1, 2, 3, 2, 1, 2, 1, 1,
       3, 3, 1, 1, 1, 1, 1, 3, 3, 1, 3, 1, 3, 1, 3, 3, 3, 3, 3, 1, 3, 3,
       2, 1, 0, 3, 3, 3, 3, 1, 3, 2, 1, 3, 3, 3, 3, 3, 0, 3, 3, 3, 1, 2,
       1, 1, 1, 1, 1, 3, 1, 3, 1, 1, 3, 3, 1, 3, 3, 3, 1, 3, 3, 1, 1, 3,
       3, 3, 0, 3, 3, 3, 0, 3, 3, 3, 1, 2, 3, 3, 1, 3, 3, 1, 3, 1, 3, 3,
       1, 3, 3, 3, 3, 1, 1, 2, 3, 2, 3, 3, 2, 3, 3, 3, 1, 3, 3, 3, 3, 3,
       3, 3, 1, 1, 1, 1, 3, 3, 3, 1, 3, 1, 3, 3, 1, 3, 3, 1, 1, 3, 3, 3,
       3, 1, 1, 1, 1, 2, 1, 3, 1, 1, 3, 1, 0, 1, 3, 1, 3, 2, 3, 0, 0, 3,
       3, 3, 3, 3, 0, 3, 1, 3, 3, 1, 3, 1, 3, 3, 3, 1, 3, 1, 1, 1, 1, 3,
       1, 1, 0, 3, 3, 3, 3, 3, 3, 3, 2, 1, 3, 2, 3, 1, 0, 1, 3, 3, 1, 3,
       1, 3, 3, 1, 1, 1, 3, 1, 3, 2, 3, 3, 0, 1, 3, 3, 3, 1, 1, 3, 3, 3,
       3, 2, 3, 3, 3, 3, 1, 3, 3, 3, 3, 0, 1, 1, 2, 1, 1, 3, 3, 3, 1, 1,
       2, 3])

In [11]:
y_train

array([3, 1, 1, 3, 3, 3, 3, 3, 1, 3, 3, 0, 3, 1, 3, 1, 1, 3, 1, 0, 0, 1,
       1, 0, 0, 3, 1, 2, 1, 1, 1, 3, 1, 1, 3, 1, 1, 1, 3, 3, 0, 1, 3, 1,
       3, 1, 3, 3, 3, 1, 2, 3, 0, 1, 0, 3, 3, 1, 3, 1, 2, 3, 3, 3, 1, 3,
       3, 1, 3, 1, 2, 3, 3, 2, 1, 3, 1, 1, 2, 2, 0, 1, 3, 3, 1, 3, 2, 2,
       1, 1, 0, 3, 1, 3, 1, 3, 1, 3, 3, 0, 2, 3, 1, 0, 3, 3, 3, 1, 0, 2,
       1, 1, 1, 2, 3, 3, 1, 1, 3, 3, 1, 3, 2, 3, 3, 2, 1, 1, 1, 1, 3, 3,
       1, 1, 3, 3, 1, 3, 2, 2, 0, 3, 3, 2, 3, 2, 0, 3, 1, 3, 1, 3, 3, 3,
       3, 1, 1, 1, 1, 1, 2, 3, 3, 1, 2, 1, 3, 3, 3, 2, 1, 1, 3, 1, 0, 1,
       3, 3, 3, 1, 1, 3, 1, 3, 3, 3, 3, 3, 2, 3, 1, 3, 1, 1, 3, 2, 2, 1,
       1, 2, 1, 1, 1, 3, 3, 3, 2, 3, 3, 1, 1, 3, 1, 2, 0, 1, 3, 1, 3, 3,
       1, 2, 3, 2, 1, 2, 3, 1, 1, 3, 1, 1, 0, 0, 2, 3, 1, 3, 1, 1, 3, 3,
       2, 3, 1, 1, 2, 1, 1, 1, 3, 1, 1, 1, 1, 3, 1, 2, 3, 1, 1, 3, 3, 1,
       0, 3, 1, 3, 2, 2, 3, 0, 3, 0, 0, 3, 0, 3, 1, 0, 1, 1, 3, 3, 0, 3,
       0, 1, 3, 0, 3, 3, 1, 0, 3, 3, 1, 1, 1, 0, 3,

In [12]:
dict_names = np.array(["LALV", "HALV", "LAHV", "HAHV"])
dict_names[y_pred_test]

array(['HAHV', 'HALV', 'HALV', 'HAHV', 'HALV', 'HAHV', 'HALV', 'HALV',
       'HAHV', 'HALV', 'HAHV', 'HALV', 'HALV', 'HAHV', 'HALV', 'HALV',
       'HAHV', 'HAHV', 'HAHV', 'HALV', 'HAHV', 'HALV', 'HAHV', 'HAHV',
       'HAHV', 'HAHV', 'HAHV', 'HAHV', 'LAHV', 'HALV', 'HAHV', 'HALV',
       'HALV', 'HALV', 'HAHV', 'HAHV', 'HALV', 'LAHV', 'HAHV', 'LAHV',
       'HALV', 'LAHV', 'HALV', 'HALV', 'HAHV', 'HAHV', 'HALV', 'HALV',
       'HALV', 'HALV', 'HALV', 'HAHV', 'HAHV', 'HALV', 'HAHV', 'HALV',
       'HAHV', 'HALV', 'HAHV', 'HAHV', 'HAHV', 'HAHV', 'HAHV', 'HALV',
       'HAHV', 'HAHV', 'LAHV', 'HALV', 'LALV', 'HAHV', 'HAHV', 'HAHV',
       'HAHV', 'HALV', 'HAHV', 'LAHV', 'HALV', 'HAHV', 'HAHV', 'HAHV',
       'HAHV', 'HAHV', 'LALV', 'HAHV', 'HAHV', 'HAHV', 'HALV', 'LAHV',
       'HALV', 'HALV', 'HALV', 'HALV', 'HALV', 'HAHV', 'HALV', 'HAHV',
       'HALV', 'HALV', 'HAHV', 'HAHV', 'HALV', 'HAHV', 'HAHV', 'HAHV',
       'HALV', 'HAHV', 'HAHV', 'HALV', 'HALV', 'HAHV', 'HAHV', 'HAHV',
      

In [17]:
from sklearn.model_selection import train_test_split
X_train_sub, X_test_sub, y_train_sub, y_test_sub = train_test_split(X_train, y_train, random_state=10)

In [18]:
clf.fit(
    X_train_sub, y_train_sub, #X_train=X_train, y_train=y_train,
    #eval_set=[(X_train, y_train), (X_val, y_val)],
    #eval_name=['train', 'valid'],
    #eval_metric=['auc'],
    max_epochs=max_epochs, patience=20,
    batch_size=1024, virtual_batch_size=128,
    num_workers=0,
    weights=1,
    drop_last=False
)

No early stopping will be performed, last training weights will be used.
epoch 0  | loss: 1.25786 |  0:00:00s
epoch 1  | loss: 0.69227 |  0:00:00s
epoch 2  | loss: 0.50939 |  0:00:00s
epoch 3  | loss: 0.57487 |  0:00:00s
epoch 4  | loss: 0.41998 |  0:00:00s
epoch 5  | loss: 0.41619 |  0:00:00s
epoch 6  | loss: 0.40817 |  0:00:00s
epoch 7  | loss: 0.22652 |  0:00:00s
epoch 8  | loss: 0.31879 |  0:00:00s
epoch 9  | loss: 0.23128 |  0:00:00s
epoch 10 | loss: 0.26739 |  0:00:00s
epoch 11 | loss: 0.15415 |  0:00:00s
epoch 12 | loss: 0.26912 |  0:00:00s
epoch 13 | loss: 0.25171 |  0:00:01s
epoch 14 | loss: 0.20351 |  0:00:01s
epoch 15 | loss: 0.18181 |  0:00:01s
epoch 16 | loss: 0.20005 |  0:00:01s
epoch 17 | loss: 0.14656 |  0:00:01s
epoch 18 | loss: 0.13071 |  0:00:01s
epoch 19 | loss: 0.17443 |  0:00:01s
epoch 20 | loss: 0.1118  |  0:00:01s
epoch 21 | loss: 0.09227 |  0:00:01s
epoch 22 | loss: 0.13599 |  0:00:01s
epoch 23 | loss: 0.10216 |  0:00:01s
epoch 24 | loss: 0.08621 |  0:00:01s
ep

In [19]:
from sklearn import metrics
y_pred_sub = clf.predict(X_test_sub)

print(f"Classification report:\n"
f"{metrics.f1_score(y_test_sub, y_pred_sub, average='micro')}\n")

Classification report:
0.7745664739884393



In [16]:
test.shape

(288, 317)

In [34]:
#import tenso
#callback = keras.callbacks.EarlyStopping(monitor='loss', patience=3)
clf.fit(
    X_train_sub, y_train_sub, #X_train=X_train, y_train=y_train,
    eval_set=[(X_train, y_train), (X_test_sub, y_test_sub)],
    eval_name=['train', 'valid'],
    eval_metric=['accuracy'],
    max_epochs=max_epochs, patience=20,
    batch_size=1024, virtual_batch_size=128,
    num_workers=0,
    weights=1,
    drop_last=False,
    #callbacks=[callback]
)

epoch 0  | loss: 0.0089  | train_accuracy: 0.94203 | valid_accuracy: 0.77457 |  0:00:00s
epoch 1  | loss: 0.01648 | train_accuracy: 0.93768 | valid_accuracy: 0.76301 |  0:00:00s
epoch 2  | loss: 0.02852 | train_accuracy: 0.94203 | valid_accuracy: 0.78035 |  0:00:00s
epoch 3  | loss: 0.07128 | train_accuracy: 0.93623 | valid_accuracy: 0.75145 |  0:00:00s
epoch 4  | loss: 0.01788 | train_accuracy: 0.93333 | valid_accuracy: 0.74566 |  0:00:00s
epoch 5  | loss: 0.06267 | train_accuracy: 0.93188 | valid_accuracy: 0.73988 |  0:00:00s
epoch 6  | loss: 0.02966 | train_accuracy: 0.92899 | valid_accuracy: 0.74566 |  0:00:01s
epoch 7  | loss: 0.031   | train_accuracy: 0.92754 | valid_accuracy: 0.73988 |  0:00:01s
epoch 8  | loss: 0.03514 | train_accuracy: 0.92899 | valid_accuracy: 0.73988 |  0:00:01s
epoch 9  | loss: 0.02431 | train_accuracy: 0.92609 | valid_accuracy: 0.71676 |  0:00:01s
epoch 10 | loss: 0.03672 | train_accuracy: 0.92754 | valid_accuracy: 0.71676 |  0:00:01s
epoch 11 | loss: 0.02

In [42]:
val = pd.read_csv('/home/nghia/PR_LAB/KERC_Chall/kerc2021/KERC21Dataset/KERC21Dataset/val_Tab.csv')
test = pd.read_csv('/home/nghia/PR_LAB/KERC_Chall/kerc2021/KERC21Dataset/KERC21Dataset/test_Tab.csv')

submission_df = pd.concat([val, test])
submission_df

,ID,0,1,2,3,4,5,6,7,8,...,306,307,308,309,310,311,312,313,314,315
0,22Nj7cPW,-0.019885,0.046958,0.733678,0.618324,1.042693,1.461745,1.664820,1.746297,-8.286156,...,-0.610064,3.167050,3.720327,1.201704,-0.602058,0.000000,0.000000,0.000000,3.990296,0.601049
1,2A9QeqgD,-0.042184,0.064785,0.963826,0.708307,1.132032,1.538334,1.881786,1.995223,-6.215798,...,-0.038016,-5.161601,-1.881224,0.272850,0.268077,0.000000,0.000000,0.000000,0.219902,0.014584
2,2AwkKhSz,0.057305,0.031962,0.676590,0.631667,1.168806,1.613567,1.975880,2.019866,-6.183857,...,0.546570,-4.731420,-1.383980,0.791501,0.495659,0.000000,0.000000,0.000000,0.288689,0.018951
3,2SEMzqof,-0.072864,0.087234,1.031330,0.284738,0.542283,0.845110,1.182093,1.357307,-7.952219,...,-0.393127,-1.146166,2.782749,1.507315,0.300281,-0.161241,1.992280,0.663413,-0.173976,-0.694309
4,2Sq686TD,0.000000,0.000000,0.000000,0.103822,0.007811,-0.004460,-0.006873,-0.003280,-0.005154,...,0.740072,-5.031872,-4.574648,-5.549634,-5.120717,-2.138377,2.276388,-4.976772,-4.874982,1.915179
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
283,KiSPDRg8,0.087915,0.021209,0.637186,0.336705,0.682841,1.051483,1.254413,1.612583,-8.117909,...,0.307423,-0.472953,-5.095288,-2.226338,0.168159,-0.318635,3.054167,1.812393,1.226686,0.554353
284,bKHxJjeg,0.001115,0.001182,0.009039,0.003976,0.007124,0.003039,0.004125,0.000028,0.000154,...,1.761269,-5.876279,-5.192789,-5.600039,-1.889896,-5.850311,2.347711,-4.213875,-3.215955,4.853327
285,3PNiPgms,-0.005370,0.000922,-0.002394,-0.003417,-0.007401,-0.009789,-0.008063,0.001677,-0.008518,...,1.288340,-7.018189,-6.288341,-6.838863,-5.384285,-3.139098,1.919908,-5.632252,-5.086109,1.778868
286,jyUDNewE,0.000000,0.000000,0.000000,1.520619,0.104700,-0.201841,0.049185,0.739341,0.558004,...,1.878504,1.319542,-2.443841,1.110149,2.935334,2.289370,-2.546097,0.957361,2.796655,2.283839


In [36]:
nunique = submission_df.nunique()
types = submission_df.dtypes

categorical_columns = []
categorical_dims = {}

for col in submission_df.columns:
    l_enc = LabelEncoder()
    submission_df[col] = l_enc.fit_transform(submission_df[col].values)
    categorical_columns.append(col)
    categorical_dims[col] = len(l_enc.classes_)

fea_test_list = [x for x in submission_df.columns.to_list() if x not in ["ID"]]
features = [col for col in submission_df[fea_test_list]]
cat_idxs = [i for i, f in enumerate(features) if f in categorical_columns]
cat_dims = [ categorical_dims[f] for i, f in enumerate(features) if f in categorical_columns]
len(cat_dims), len(cat_idxs)

(316, 316)

In [37]:
X_submit = submission_df[features].values

y_submit_pred = clf.predict(X_submit)
y_submit_pred

array([1, 1, 3, 1, 1, 2, 3, 3, 3, 2, 3, 3, 3, 3, 3, 3, 1, 3, 3, 3, 1, 3,
       1, 3, 1, 1, 2, 2, 3, 1, 3, 1, 0, 2, 1, 0, 1, 3, 3, 3, 2, 3, 0, 0,
       3, 3, 1, 0, 2, 1, 2, 3, 3, 3, 0, 3, 3, 1, 3, 3, 3, 3, 1, 1, 1, 0,
       1, 3, 1, 2, 3, 3, 1, 1, 3, 1, 1, 1, 2, 1, 3, 3, 3, 2, 3, 3, 3, 1,
       1, 0, 1, 3, 0, 3, 3, 3, 3, 3, 2, 2, 1, 3, 2, 2, 0, 0, 1, 2, 1, 1,
       3, 0, 3, 0, 2, 2, 1, 1, 1, 1, 3, 3, 1, 0, 3, 0, 3, 1, 0, 0, 0, 3,
       3, 3, 3, 1, 3, 1, 1, 3, 3, 2, 2, 1, 2, 3, 3, 1, 1, 0, 0, 1, 1, 3,
       3, 3, 1, 3, 2, 1, 2, 3, 3, 3, 1, 3, 1, 1, 1, 3, 3, 1, 1, 3, 1, 3,
       1, 3, 3, 3, 3, 3, 3, 1, 2, 3, 3, 0, 3, 1, 1, 3, 3, 0, 3, 2, 1, 1,
       3, 3, 3, 3, 1, 3, 2, 0, 0, 3, 3, 3, 3, 0, 0, 1, 3, 3, 1, 3, 1, 0,
       3, 2, 3, 2, 3, 3, 3, 3, 2, 1, 1, 3, 3, 1, 3, 1, 1, 1, 0, 3, 3, 3,
       3, 3, 1, 3, 3, 1, 2, 0, 3, 1, 1, 2, 1, 3, 1, 1, 1, 2, 2, 2, 3, 2,
       3, 2, 1, 0, 1, 2, 2, 3, 3, 0, 3, 0, 3, 3, 2, 1, 0, 3, 2, 0, 3, 3,
       3, 1, 3, 1, 3, 2, 0, 3, 2, 0, 3, 1, 3, 2, 1,

In [38]:
dict_names = np.array(["HAHV", "HALV", "LAHV", "LALV"])
t = dict_names[y_submit_pred]
t


array(['HALV', 'HALV', 'LALV', 'HALV', 'HALV', 'LAHV', 'LALV', 'LALV',
       'LALV', 'LAHV', 'LALV', 'LALV', 'LALV', 'LALV', 'LALV', 'LALV',
       'HALV', 'LALV', 'LALV', 'LALV', 'HALV', 'LALV', 'HALV', 'LALV',
       'HALV', 'HALV', 'LAHV', 'LAHV', 'LALV', 'HALV', 'LALV', 'HALV',
       'HAHV', 'LAHV', 'HALV', 'HAHV', 'HALV', 'LALV', 'LALV', 'LALV',
       'LAHV', 'LALV', 'HAHV', 'HAHV', 'LALV', 'LALV', 'HALV', 'HAHV',
       'LAHV', 'HALV', 'LAHV', 'LALV', 'LALV', 'LALV', 'HAHV', 'LALV',
       'LALV', 'HALV', 'LALV', 'LALV', 'LALV', 'LALV', 'HALV', 'HALV',
       'HALV', 'HAHV', 'HALV', 'LALV', 'HALV', 'LAHV', 'LALV', 'LALV',
       'HALV', 'HALV', 'LALV', 'HALV', 'HALV', 'HALV', 'LAHV', 'HALV',
       'LALV', 'LALV', 'LALV', 'LAHV', 'LALV', 'LALV', 'LALV', 'HALV',
       'HALV', 'HAHV', 'HALV', 'LALV', 'HAHV', 'LALV', 'LALV', 'LALV',
       'LALV', 'LALV', 'LAHV', 'LAHV', 'HALV', 'LALV', 'LAHV', 'LAHV',
       'HAHV', 'HAHV', 'HALV', 'LAHV', 'HALV', 'HALV', 'LALV', 'HAHV',
      

In [43]:
#make submission
#id = submission_df['ID'].copy()
predict = t
submit_infor = {
    "Id": submission_df['ID'],
    "Predicted": predict
}
submit_final = pd.DataFrame(submit_infor)

In [40]:
#submission_df

In [44]:
submit_final


,Id,Predicted
0,22Nj7cPW,HALV
1,2A9QeqgD,HALV
2,2AwkKhSz,LALV
3,2SEMzqof,HALV
4,2Sq686TD,HALV
...,...,...
283,KiSPDRg8,HALV
284,bKHxJjeg,LALV
285,3PNiPgms,LALV
286,jyUDNewE,LALV


In [45]:
submit_final.to_csv('logs/submission_val_test.csv')


In [46]:
clf.save_model('TabNet1.h5')

Successfully saved model at TabNet1.h5.zip


'TabNet1.h5.zip'